Looking at making an algorithm which can take user inputs for their risk profile and what product they want to launch. Then we want to calculate an estimate for the elasticity and generate a scroe out of 100 (essentially chances of success).
Will look at machine learning to predict values and compare to actual values (Random Forest or XgBoost or Neural Network)
The first thing to do is to import our dataset (Fake dataset for testing).

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

df = pd.read_csv('/Users/nickking/Desktop/Documents/Code/Sales/fake_sales_data_50k.csv')
df.head()

,transaction_id,date,country,region,product_category,product_subcategory,product_id,brand,price,quantity_sold,...,discount_rate,marketing_spend,season,holiday,customer_age,customer_income,customer_loyalty_score,online_sale,return_rate,stock_level
0,1,2024-11-04,South Africa,Western Cape,Clothing,T-shirt,CLO_T-S_1769,BrandE,750.33,72,...,0.18,780.09,Summer,0,28,4862.86,99,1,0.14,149
1,2,2024-07-06,Canada,British Columbia,Home Appliances,Toaster,HOM_TOA_1189,BrandE,1444.55,2,...,0.05,917.02,Summer,0,39,567.13,88,0,0.10,169
2,3,2024-01-14,Canada,Quebec,Sports,Tennis Racket,SPO_TEN_9433,BrandC,183.03,55,...,0.29,2333.81,Summer,1,24,2119.98,38,1,0.19,315
3,4,2024-01-02,Germany,Berlin,Clothing,Shoes,CLO_SHO_2021,BrandD,772.06,2,...,0.21,2200.76,Spring,0,25,826.69,77,0,0.05,359
4,5,2024-08-02,Germany,Hamburg,Home Appliances,Vacuum,HOM_VAC_2016,BrandC,1041.10,54,...,0.01,4211.42,Summer,0,27,9000.86,13,0,0.18,455


Just checking all our columns so we can see every variable which may affect elasticity

In [3]:
df.columns

Index(['transaction_id', 'date', 'country', 'region', 'product_category',
       'product_subcategory', 'product_id', 'brand', 'price', 'quantity_sold',
       'competitor_price', 'discount_rate', 'marketing_spend', 'season',
       'holiday', 'customer_age', 'customer_income', 'customer_loyalty_score',
       'online_sale', 'return_rate', 'stock_level'],
      dtype='object')

Taking off our identifiers as it is not useful for the analysis, sorting by date and creating price lags and rolling averages

In [4]:
df = df.drop('transaction_id', axis=1)

df = df.sort_values(["product_subcategory", "brand", "country", "date"]).reset_index(drop=True)

df["price_rollmean7"] = (
    df.groupby(["product_subcategory", "brand", "country"])["price"]
      .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)

df["quantity_rollmean7"] = (
    df.groupby(["product_subcategory", "brand", "country"])["quantity_sold"]
      .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)

df["price_lag1"] = df["price"].shift(1)
df["quantity_lag1"] = df["quantity_sold"].shift(1)

brand_map = df.groupby("brand")["quantity_sold"].mean().to_dict()
df["brand_encoded"] = df["brand"].map(brand_map)

df = df.dropna().reset_index(drop=True)
df.head()

,date,country,region,product_category,product_subcategory,product_id,brand,price,quantity_sold,competitor_price,...,customer_income,customer_loyalty_score,online_sale,return_rate,stock_level,price_rollmean7,quantity_rollmean7,price_lag1,quantity_lag1,brand_encoded
0,2024-01-18,Australia,Victoria,Sports,Basketball,SPO_BAS_6660,BrandA,924.95,41,944.52,...,6487.49,29,1,0.03,172,1098.670000,69.500000,1272.39,98.0,50.400545
1,2024-01-20,Australia,Queensland,Sports,Basketball,SPO_BAS_6094,BrandA,1141.08,38,1126.18,...,1831.85,66,0,0.13,304,1112.806667,59.000000,924.95,41.0,50.400545
2,2024-01-22,Australia,Victoria,Sports,Basketball,SPO_BAS_9563,BrandA,1800.06,42,1857.54,...,8959.81,59,0,0.09,415,1284.620000,54.750000,1141.08,38.0,50.400545
3,2024-01-31,Australia,Queensland,Sports,Basketball,SPO_BAS_4364,BrandA,387.05,93,351.45,...,6371.75,43,1,0.08,80,1105.106000,62.400000,1800.06,42.0,50.400545
4,2024-02-14,Australia,Queensland,Sports,Basketball,SPO_BAS_3226,BrandA,964.89,98,964.50,...,815.62,28,0,0.10,195,1081.736667,68.333333,387.05,93.0,50.400545


Now - want to see all our information on the variables i.e. if they're numerical

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    49999 non-null  object 
 1   country                 49999 non-null  object 
 2   region                  49999 non-null  object 
 3   product_category        49999 non-null  object 
 4   product_subcategory     49999 non-null  object 
 5   product_id              49999 non-null  object 
 6   brand                   49999 non-null  object 
 7   price                   49999 non-null  float64
 8   quantity_sold           49999 non-null  int64  
 9   competitor_price        49999 non-null  float64
 10  discount_rate           49999 non-null  float64
 11  marketing_spend         49999 non-null  float64
 12  season                  49999 non-null  object 
 13  holiday                 49999 non-null  int64  
 14  customer_age            49999 non-null

Brand strength is often a key determinant of price elasticity of demand - strong brands can typically sustain higher prices without losing as many customers, while weaker brands face greater sensitivity to price changes.

However, brand is stored as an object (categorical string) in the dataset, and we cannot use it directly in regression or machine learning models without encoding it. Simply assigning arbitrary numeric IDs to brands would be meaningless because the numbers would imply a false ranking.

Instead, we should quantify brand strength using measurable proxies, such as:

    1.Historical sales performance (average quantity sold per time period controlling for price).
    2.Price premium (average price relative to competitors in the same category/region).
    3.Market share in the category/region.
    4.Customer loyalty (repeat purchase rate, churn rate if available).
    5.Advertising spend as a percentage of category sales.

Once we have one or more of these proxies, we can:
Calculate a Brand Strength Index (BSI) on a 0–1 or 0–100 scale.
Include this numerical brand strength in the feature set for elasticity estimation (Random Forest, log-log regression, etc.).
Optionally, interact brand strength with price in the model to see how much it shifts elasticity.
This way, brand becomes a measurable, numeric driver in the model rather than just a categorical label.

From our dataset we can make a BSI from Average Sales Volume, Price Premium, Loyalty Score, and Marketing Spend.

In [6]:
brand_stats = df.groupby(["brand", "product_subcategory", "country"]).agg({
    "quantity_rollmean7": "mean",
    "price_rollmean7": "mean",
    "customer_loyalty_score": "mean",
    "marketing_spend": "mean"
}).reset_index()

# Compute price premium
category_avg = df.groupby(["product_subcategory", "country"])["price_rollmean7"].mean().reset_index()
brand_stats = brand_stats.merge(category_avg, on=["product_subcategory", "country"], suffixes=("", "_cat_avg"))
brand_stats["price_premium"] = brand_stats["price_rollmean7"] / brand_stats["price_rollmean7_cat_avg"]

# Normalise metrics
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
metrics = ["quantity_rollmean7", "price_premium", "customer_loyalty_score", "marketing_spend"]
brand_stats[metrics] = scaler.fit_transform(brand_stats[metrics])

# Final BSI
brand_stats["BSI"] = brand_stats[metrics].mean(axis=1) * 100

# Merge back
df = df.merge(brand_stats[["brand", "product_subcategory", "country", "BSI"]],
              on=["brand", "product_subcategory", "country"],
              how="left")


Now we have calculated BSI -we want to find the actual elasticity for each good. This is best done by an OLS log-log regression. This works as it is a % change reacting to a 1% change. So the coefficient is the elasticity by definition.

In [7]:
import numpy as np
import statsmodels.api as sm

def calculate_elasticity(df):
    results = []

    # Loop over brand–subcategory combinations
    for (brand, subcat), group in df.groupby(["brand", "product_subcategory"]):
        # Avoid invalid values for log
        group = group[(group["price"] > 0) & (group["quantity_sold"] > 0)]
        
        if len(group) < 5:  # not enough data points
            continue
        
        # log transform
        group["log_price"] = np.log(group["price"])
        group["log_quantity"] = np.log(group["quantity_sold"])
        
        # OLS regression
        X = sm.add_constant(group["log_price"])
        y = group["log_quantity"]
        model = sm.OLS(y, X).fit()
        
        elasticity = model.params["log_price"]  # β
        results.append({
            "brand": brand,
            "product_subcategory": subcat,
            "elasticity": elasticity,
            "n_obs": len(group),
            "r2": model.rsquared
        })
    
    return pd.DataFrame(results)

# Run it
elasticity_df = calculate_elasticity(df)
print(elasticity_df.head())


    brand product_subcategory  elasticity  n_obs        r2
0  BrandA          Basketball   -0.063554    496  0.004773
1  BrandA             Blender    0.027004    504  0.000897
2  BrandA           Chocolate   -0.025751    489  0.000778
3  BrandA              Coffee    0.008211    469  0.000078
4  BrandA            Football    0.096378    459  0.010955


Now we have calculated brand strength - we want to separate our variables into numeric and non numeric

In [ ]:
y = df["quantity_sold"]

num_features = [
    "price", "competitor_price", "discount_rate", "marketing_spend",
    "customer_age", "customer_income", "customer_loyalty_score",
    "return_rate", "stock_level", "price_rollmean7", "quantity_rollmean7",
    "price_lag1", "quantity_lag1", "BSI"
]

cat_features = ["brand", "product_category", "product_subcategory", "country"]

In [ ]:
x = df[num_features + cat_features]

We want to first look at a random forest for evaluating elasticity

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

We want to predict quantity sold to determine the elasticity.

Splitting into 80-20 (can change)

In [ ]:
train_size = 0.8
split_idx = int(len(df) * train_size)
x_train = x.iloc[:split_idx]
y_train = y.iloc[:split_idx]
x_test = x.iloc[split_idx:]
y_test = y.iloc[split_idx:]


Preprocessing

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
    ]
)

Fitting the model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(
        n_estimators=200,
        max_depth=None,
        random_state=42,
        n_jobs=-1
    ))
])

rf.fit(x_train, y_train)

Predictions

In [ ]:
y_train_pred = rf.predict(x_train)
y_test_pred = rf.predict(x_test)

Printing test scores

In [ ]:
from sklearn.metrics import r2_score

train_score = r2_score(y_train, rf.predict(x_train))
test_score = r2_score(y_test, rf.predict(x_test))

print(f"Train R²: {train_score:.3f}")
print(f"Test R²: {test_score:.3f}")